# BLOOM Fine Training with Huggingface Trainer

* [HuggingFace-FineTuning](https://www.kaggle.com/code/mamunsust12/huggingface-finetuning)
* [Fine-tuning BLOOM for Summarization with Trainer API](https://huggingface.co/bigscience/bloom/discussions/234)


In [2]:
! pip install torch transformers datasets evaluate scikit-learn rouge rouge-score promptsource --quiet

In [3]:
import re
from typing import (
    List,
    Dict,
    Callable,
)
import multiprocessing

import numpy as np
import pandas as pd
from datasets import (
    load_dataset,
    get_dataset_split_names
)
import torch
from transformers import (
    AutoTokenizer,
    AutoModel,
    AutoModelForCausalLM,
    DataCollatorWithPadding,
    DataCollatorForSeq2Seq,
    BloomForCausalLM,
    TrainingArguments,
    Trainer
)
import evaluate
from promptsource.templates import (
    DatasetTemplates,
    Template
)

# Environment

In [4]:
NUM_CPUS: int = multiprocessing.cpu_count()

# Constant

In [ ]:
## Huggingface Datasets
DATASET_NAME: str = "xsum"
DATASET_TRAIN_NUM_ROWS: int = 204045      # Number of rows in the original train dataset
DATASET_STREAMING: bool = False                    # If using Dataset streaming
DATASET_TRAIN_NUM_SELECT: int = 1024       # Number of rows to use for training
DATASET_VALIDATE_NUM_SELECT: int =64

# Huggingface Tokenizer (BLOOM default token length is 2048)
MAX_TOKEN_LENGTH: int = 384         # Max token length to avoid out of memory
PER_DEVICE_BATCH_SIZE: int = 1       # GPU batch size

# Huggingface Model
# MODEL = "bigscience/bloomz-560m"
MODEL = "bigscience/bloom-560m"
USE_FLOAT16: bool = True

# Training
NUM_EPOCHS: int = 3
MAX_STEPS: int = NUM_EPOCHS * DATASET_TRAIN_NUM_SELECT if DATASET_STREAMING else -1

## Load dataset

Use [xsum](https://huggingface.co/datasets/xsum) which has PromptSource template 

<img src="./image/xsum.png" align="left" width=600/>

<img src="./image/xsum_promptsource_templates.png" align="left"/>

In [6]:
get_dataset_split_names(path=DATASET_NAME)

['train', 'validation', 'test']

In [7]:
train = load_dataset("xsum", split="train", streaming=DATASET_STREAMING)
train

Found cached dataset xsum (/root/.cache/huggingface/datasets/xsum/default/1.2.0/082863bf4754ee058a5b6f6525d0cb2b18eadb62c7b370b095d1364050a52b71)


Dataset({
    features: ['document', 'summary', 'id'],
    num_rows: 204045
})

There are two fields that you'll want to use:

- `document`: the text of the news.
- `summary`: a condensed version of `document` which'll be the model target.

In [8]:
example: Dict[str, str]  = list(train.take(1))[0] if DATASET_STREAMING else train[0]

# Prompt Template

In [9]:
prompt_templates = DatasetTemplates( dataset_name=DATASET_NAME)  
prompt_templates.all_template_names

['DOC_boils_down_to_simple_idea_that',
 'DOC_given_above_write_one_sentence',
 'DOC_how_would_you_rephrase_few_words',
 'DOC_tldr',
 'DOC_write_summary_of_above',
 'article_DOC_summary',
 'college_roommate_asked_DOC_so_I_recap',
 'read_below_DOC_write_abstract',
 'summarize_DOC',
 'summarize_this_DOC_summary']

In [10]:
template: Template = prompt_templates['summarize_DOC']
print(template.jinja)

Summarize: {{document}}|||
{{summary}}


In [11]:
prompt, response = template.apply(example=example, truncate=False)
print('-' * 80)
print("Prompt")
print('-' * 80)
print(re.sub(r'[\s\'\"]+', ' ', prompt))

print('-' * 80)
print("Response")
print('-' * 80)
print(re.sub(r'[\s\'\"]+', ' ', response))

--------------------------------------------------------------------------------
Prompt
--------------------------------------------------------------------------------
Summarize: The full cost of damage in Newton Stewart, one of the areas worst affected, is still being assessed. Repair work is ongoing in Hawick and many roads in Peeblesshire remain badly affected by standing water. Trains on the west coast mainline face disruption due to damage at the Lamington Viaduct. Many businesses and householders were affected by flooding in Newton Stewart after the River Cree overflowed into the town. First Minister Nicola Sturgeon visited the area to inspect the damage. The waters breached a retaining wall, flooding many commercial properties on Victoria Street - the main shopping thoroughfare. Jeanette Tate, who owns the Cinnamon Cafe which was badly affected, said she could not fault the multi-agency response once the flood hit. However, she said more preventative work could have been carrie

---
# Preprocess

To apply the preprocessing function over the entire dataset, use Datasets [map](https://huggingface.co/docs/datasets/main/en/package_reference/main_classes#datasets.Dataset.map) method. You can speed up the `map` function by setting `batched=True` to process multiple elements of the dataset at once.

* [Datasets - select / filter](https://huggingface.co/docs/datasets/process#select-and-filter)
* [Datasets - select](https://huggingface.co/docs/datasets/v2.11.0/en/package_reference/main_classes#datasets.Dataset.select)

### Framework Tensor Format

* [Use with PyTorch - Dataset Format](https://huggingface.co/docs/datasets/use_with_pytorch)
> By default, datasets return regular python objects: integers, floats, strings, lists, etc. To get PyTorch tensors instead, you can set the format of the dataset to pytorch using Dataset.with_format():

```
ds = ds.with_format("torch")
```

* [Using Datasets with TensorFlow](https://huggingface.co/docs/datasets/use_with_tensorflow)

> By default, datasets return regular Python objects: integers, floats, strings, lists, etc. To get TensorFlow tensors instead, you can set the format of the dataset to tf:

```
ds = ds.with_format("tf")
```

The preprocessing function you want to create needs to:

1. Prefix the input with a prompt so T5 knows this is a summarization task. Some models capable of multiple NLP tasks require prompting for specific tasks.
2. Use the keyword `text_target` argument when tokenizing labels.
3. Truncate sequences to be no longer than the maximum length set by the `max_length` parameter.

In [12]:
tokenizer = AutoTokenizer.from_pretrained(MODEL, use_fast=True)

In [13]:
def get_convert_to_request_response(template: Template) -> Callable:
    def _convert_to_prompt_response(example: Dict[str, str]) -> Dict[str, str]:
        """Generate prompt, response as a dictionary:
        {
            "prompt": "Summarize: ...",
            "response": "..."
        }

        NOTE: DO NOT use with dataset map function( batched=True). Use batch=False

        Args:
            example: single {document, summary} pair to be able to apply template
        Returns: a dictionary of pro
        """
        # assert isinstance(example, dict), f"expected dict but {type(example)}.\n{example}"
        assert isinstance(example['document'], str), f"expected str but {type(example['document'])}."
        prompt, response = template.apply(example=example, truncate=False)
        return {
            "prompt": re.sub(r'[\s\'\"]+', ' ', prompt),
            "response": re.sub(r'[\s\'\"]+', ' ', response)
        }

    return _convert_to_prompt_response

convert_to_request_response: Callable = get_convert_to_request_response(template=template)

In [14]:
convert_to_request_response(example=example)

{'prompt': 'Summarize: The full cost of damage in Newton Stewart, one of the areas worst affected, is still being assessed. Repair work is ongoing in Hawick and many roads in Peeblesshire remain badly affected by standing water. Trains on the west coast mainline face disruption due to damage at the Lamington Viaduct. Many businesses and householders were affected by flooding in Newton Stewart after the River Cree overflowed into the town. First Minister Nicola Sturgeon visited the area to inspect the damage. The waters breached a retaining wall, flooding many commercial properties on Victoria Street - the main shopping thoroughfare. Jeanette Tate, who owns the Cinnamon Cafe which was badly affected, said she could not fault the multi-agency response once the flood hit. However, she said more preventative work could have been carried out to ensure the retaining wall did not fail. It is difficult but I do think there is so much publicity for Dumfries and the Nith - and I totally apprecia

In [15]:
def tokenize_prompt_response(examples):
    """Generate the model inputs in the dictionary with format:
    {
        "input_ids": List[int], 
        "attention_mask": List[int]",
        "labels": List[int]
    }
    
    Note: Huggngface dataaset map(batched=True, batch_size=n) merges values of 
    n dictionarys into a values of the key. If you have n instances of {"key", "v"}, then
    you will get {"key": ["v", "v", "v", ...] }.
    
    Args:
        examples:   a dictionary of format {
            "prompt": [prompt+],
            "response": [respnse+]
        } where + means more than one instance because of Dataset.map(batched=True)
    """    
    inputs: Dict[str, List[int]] = tokenizer(
        text_target=examples["prompt"], 
        max_length=MAX_TOKEN_LENGTH, 
        truncation=True
    )

    labels: Dict[str, List[int]] = tokenizer(
        text_target=examples["response"], 
        max_length=MAX_TOKEN_LENGTH, 
        truncation=True,
        padding='max_length',
    )
    inputs["labels"] = labels["input_ids"]
    
    return inputs

In [16]:
tokenized: Dict[str, List[int]] = tokenize_prompt_response(examples=convert_to_request_response(example=example))
tokenizer.decode(token_ids=tokenized['input_ids'])

'Summarize: The full cost of damage in Newton Stewart, one of the areas worst affected, is still being assessed. Repair work is ongoing in Hawick and many roads in Peeblesshire remain badly affected by standing water. Trains on the west coast mainline face disruption due to damage at the Lamington Viaduct. Many businesses and householders were affected by flooding in Newton Stewart after the River Cree overflowed into the town. First Minister Nicola Sturgeon visited the area to inspect the damage. The waters breached a retaining wall, flooding many commercial properties on Victoria Street - the main shopping thoroughfare. Jeanette Tate, who owns the Cinnamon Cafe which was badly affected, said she could not fault the multi-agency response once the flood hit. However, she said more preventative work could have been carried out to ensure the retaining wall did not fail. It is difficult but I do think there is so much publicity for Dumfries and the Nith - and I totally appreciate that - b

## Apply preprocessing

In [17]:
remove_column_names: List[str] = list(train.features.keys())

tokenized_train = train.map(
    function=convert_to_request_response, 
    batched=False,
    batch_size=2048,
    drop_last_batch=False,
    remove_columns=remove_column_names,
).map(
    function=tokenize_prompt_response, 
    batched=True,
    batch_size=32,
    drop_last_batch=True,
    remove_columns=['prompt', 'response']
).shuffle(
    seed=42
).with_format(
    "torch"
)

if DATASET_STREAMING:
    tokenized_train = tokenized_train.take(DATASET_TRAIN_NUM_SELECT)
    print(f"size of tokenized_train: {len(tokenized_train)}")
else:
    tokenized_train = tokenized_train.select(
        indices=range(DATASET_TRAIN_NUM_SELECT)
    )

del train

Loading cached processed dataset at /root/.cache/huggingface/datasets/xsum/default/1.2.0/082863bf4754ee058a5b6f6525d0cb2b18eadb62c7b370b095d1364050a52b71/cache-369dd8c0c1bd7d59.arrow


Map:   0%|          | 0/204032 [00:00<?, ? examples/s]

In [18]:
tokenized_validation =  load_dataset(
    path="xsum", 
    split="validation", 
    streaming=DATASET_STREAMING
).map(
    function=convert_to_request_response, 
    batched=False,
    batch_size=2048,
    drop_last_batch=False,
    remove_columns=remove_column_names,
).map(
    function=tokenize_prompt_response, 
    batched=True,
    batch_size=32,
    drop_last_batch=True,
    remove_columns=['prompt', 'response']
).shuffle(
    seed=42
).with_format(
    "torch"
)

if DATASET_STREAMING:
    tokenized_validation = tokenized_validation.take(DATASET_TRAIN_NUM_SELECT)
else:
    tokenized_validation = tokenized_validation.select(
        indices=range(DATASET_VALIDATE_NUM_SELECT)
    )

Found cached dataset xsum (/root/.cache/huggingface/datasets/xsum/default/1.2.0/082863bf4754ee058a5b6f6525d0cb2b18eadb62c7b370b095d1364050a52b71)
Loading cached processed dataset at /root/.cache/huggingface/datasets/xsum/default/1.2.0/082863bf4754ee058a5b6f6525d0cb2b18eadb62c7b370b095d1364050a52b71/cache-e701ee11c489e755.arrow


Map:   0%|          | 0/11328 [00:00<?, ? examples/s]

## Verify preprocessing

In [19]:
examples = list(tokenized_train.take(2)) if DATASET_STREAMING else tokenized_train[:2]

In [20]:
print('-' * 80)
print("prompt")
print('-' * 80)

if DATASET_STREAMING:
    print(tokenizer.decode(token_ids=examples[0]['input_ids']))
else:
    print(tokenizer.decode(token_ids=examples['input_ids'][0]))

print('-' * 80)
print("response")
print('-' * 80)
if DATASET_STREAMING:
    print(tokenizer.decode(token_ids=examples[0]['labels']))
else:
    print(tokenizer.decode(token_ids=examples['labels'][0]))

--------------------------------------------------------------------------------
prompt
--------------------------------------------------------------------------------
Summarize: 10 August 2016 Last updated at 13:37 BST William Joyner was born without fingers on his left hand. The Reading FC fan s blue and white prosthetic has been made by a senior lecturer and a PHD student at the University of Bedfordshire. My friends are really excited for me, I can t wait to take it home and show everyone, he said.
--------------------------------------------------------------------------------
response
--------------------------------------------------------------------------------
<pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>

---
# Training

Regarding the hyperparameter to use, need to investivage. Those used in [Fine-tune the model? #46 by NXBY - opened Jul 16, 2022](https://huggingface.co/bigscience/bloom/discussions/46#633d452d48ab6a0add2b61bd) might be a starting point.

```
!python run_qa.py \
  --model_name_or_path bigscience/bloom-560m \
  --dataset_name squad_v2 \
  --do_train \
  --per_device_train_batch_size 12 \
  --learning_rate 3e-5 \
  --num_train_epochs 2 \
  --max_seq_length 384 \
  --doc_stride 128 \
  --output_dir /tmp/debug_seq2seq_squad/ \
  --eval_accumulation_steps 1 \
  --version_2_with_negative \
  --overwrite_output_dir
```

## Model

We may need to use a specific model class e.g.  ```AutoModelForSequenceClassification```  to use BERT for classifying pairs of sentences because BERT has not been pretrained on such a task, so the head of the pretrained model has been discarded and a new head suitable for sequence classification has been added instead in ```AutoModelForSequenceClassification```.

```
from transformers import AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)
```

Note that BLOOM is a Decoder model, not Encoder-Decoder, hence cannot be used with ```AutoModelForSeq2SeqLM``` which causes:
```
ValueError: Unrecognized configuration class <class 'transformers.models.bloom.configuration_bloom.BloomConfig'> for this kind of AutoModel: AutoModelForSeq2SeqLM. Model type should be one of BartConfig, BigBirdPegasusConfig, BlenderbotConfig, BlenderbotSmallConfig, EncoderDecoderConfig, FSMTConfig, LEDConfig, LongT5Config, M2M100Config, MarianConfig, MBartConfig, MT5Config, MvpConfig, PegasusConfig, PegasusXConfig, PLBartConfig, ProphetNetConfig, SwitchTransformersConfig, T5Config, XLMProphetNetConfig.
```

Have a solid understanding on the model architecture and the task to execute for the fine-tuning, and devise the appropriate model to use. BLOOM is still a new model and Decoder architecture such as GPT 

Note that we cannot use AutoModel as it causes the error:

```
TypeError: The current model class (BloomModel) is not compatible with `.generate()`, as it doesn't have a language model head. Please use one of the following classes instead: {'BloomForCausalLM'}
```

<img src="./image/bloom_model_classes.png" align="left" width=200/>

In [21]:
# model = AutoModelForSeq2SeqLM.from_pretrained(MODEL)
# model = AutoModel.from_pretrained(MODEL)
model = BloomForCausalLM.from_pretrained(MODEL)
model.cuda()

BloomForCausalLM(
  (transformer): BloomModel(
    (word_embeddings): Embedding(250880, 1024)
    (word_embeddings_layernorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
    (h): ModuleList(
      (0): BloomBlock(
        (input_layernorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (self_attention): BloomAttention(
          (query_key_value): Linear(in_features=1024, out_features=3072, bias=True)
          (dense): Linear(in_features=1024, out_features=1024, bias=True)
          (attention_dropout): Dropout(p=0.0, inplace=False)
        )
        (post_attention_layernorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (mlp): BloomMLP(
          (dense_h_to_4h): Linear(in_features=1024, out_features=4096, bias=True)
          (gelu_impl): BloomGelu()
          (dense_4h_to_h): Linear(in_features=4096, out_features=1024, bias=True)
        )
      )
      (1): BloomBlock(
        (input_layernorm): LayerNorm((1024,), eps=1e-05, elementw

In [22]:
type(model)

transformers.models.bloom.modeling_bloom.BloomForCausalLM

In [23]:
dir(model)

['T_destination',
 '__annotations__',
 '__call__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_apply',
 '_auto_class',
 '_backward_compatibility_gradient_checkpointing',
 '_backward_hooks',
 '_buffers',
 '_call_impl',
 '_convert_head_mask_to_5d',
 '_convert_to_bloom_cache',
 '_convert_to_standard_cache',
 '_create_repo',
 '_expand_inputs_for_generation',
 '_extract_past_from_model_output',
 '_forward_hooks',
 '_forward_pre_hooks',
 '_from_config',
 '_get_backward_hooks',
 '_get_decoder_start_token_id',
 '_get_files_timestamps',
 '_get_logits_processor',
 '_get_logits_warper',
 '_get_name',
 '_get_resized_embeddings',
 '_get_resize

In [24]:
model.config

BloomConfig {
  "_name_or_path": "bigscience/bloom-560m",
  "apply_residual_connection_post_layernorm": false,
  "architectures": [
    "BloomForCausalLM"
  ],
  "attention_dropout": 0.0,
  "attention_softmax_in_fp32": true,
  "bias_dropout_fusion": true,
  "bos_token_id": 1,
  "eos_token_id": 2,
  "hidden_dropout": 0.0,
  "hidden_size": 1024,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "masked_softmax_fusion": true,
  "model_type": "bloom",
  "n_head": 16,
  "n_inner": null,
  "n_layer": 24,
  "offset_alibi": 100,
  "pad_token_id": 3,
  "pretraining_tp": 1,
  "skip_bias_add": true,
  "skip_bias_add_qkv": false,
  "slow_but_exact": false,
  "transformers_version": "4.28.0",
  "unk_token_id": 0,
  "use_cache": true,
  "vocab_size": 250880
}

In [25]:
prompt = "Summarize: The Inflation Reduction Act lowers prescription drug costs, health care costs, and energy costs. It's the most aggressive action on tackling the climate crisis in American history, which will lift up American workers and create good-paying, union jobs across the country. It'll lower the deficit and ask the ultra-wealthy and corporations to pay their fair share. And no one making under $400,000 per year will pay a penny more in taxes."

Use the [generate()](https://huggingface.co/docs/transformers/main/en/main_classes/text_generation#transformers.GenerationMixin.generate) method to create the summarization. For more details about the different text generation strategies and parameters for controlling generation, check out the [Text Generation](https://huggingface.co/docs/transformers/main/en/tasks/../main_classes/text_generation) API.

In [26]:
def predict(prompt: str) -> str:
    inputs = tokenizer(prompt, return_tensors='pt')
    print(inputs["input_ids"].shape)
    
    response_tokens = model.generate(
        inputs["input_ids"].cuda(), 
        max_new_tokens=1,
        do_sample=False, 
        top_k=50, 
        top_p=0.9
    )[0]
    response = tokenizer.decode(response_tokens, skip_special_tokens=True)
    return response

In [27]:
print(predict(prompt=prompt))

torch.Size([1, 97])
Summarize: The Inflation Reduction Act lowers prescription drug costs, health care costs, and energy costs. It's the most aggressive action on tackling the climate crisis in American history, which will lift up American workers and create good-paying, union jobs across the country. It'll lower the deficit and ask the ultra-wealthy and corporations to pay their fair share. And no one making under $400,000 per year will pay a penny more in taxes. The


## Data Collator

Tensors going into a model must have the same shape. Hcne pad all the examples to the length of the longest element when we batch elements together — a technique we refer to as dynamic padding. We delay the padding to the last moment, otherwise we bring around padded data which waste the memory and computation time. 

The function that is responsible for packaging examples into a batch is a collate function, which you pass to a DataLoader as an argument when instantiate it. The collate function converts examples to PyTorch tensors and concatenate them (recursively if your elements are lists, tuples, or dictionaries).

The collator is [DataCollatorWithPadding(tokenizer=tokenizer)](https://huggingface.co/docs/transformers/main_classes/data_collator#transformers.DataCollatorWithPadding) that takes a tokenizer as an argument to know which padding token to use, and whether the model expects padding to be on the left or on the right.

```
from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
```

if there are eight tokenized sentences whose lengths are ```[50, 59, 47, 67, 59, 50, 62, 32]```, the collator will pad the sentences so that the length will be all 67 as ```[67,  67,  67,  67,  67,  67,  67,  67 ]```.

In [28]:
# DataCollatorWithPadding does not pad 'labels' which causes an error at train()
# https://stackoverflow.com/a/74228547/4281353
data_collator = DataCollatorWithPadding(
    tokenizer=tokenizer, 
    padding='max_length',
    pad_to_multiple_of=8,
    max_length=MAX_TOKEN_LENGTH,
    return_tensors='pt'
)

In [29]:
# data_collator = DataCollatorForSeq2Seq(
#     model=model,
#     tokenizer=tokenizer, 
#     padding='max_length',
#     pad_to_multiple_of=8,
#     max_length=MAX_TOKEN_LENGTH,
#     return_tensors='pt'
# )

In [30]:
collated = data_collator(list(tokenized_train.take(1))[0]) if DATASET_STREAMING else data_collator(tokenized_train[0])
for key in collated.keys():
    print(f"{key}:{len(collated[key])}")
    
assert len(collated['input_ids']) == len(collated['labels']), \
    f"expected the same length of input_ids:[{len(collated['input_ids'])}] and labels:{len(collated['labels'])}"

You're using a BloomTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


input_ids:384
attention_mask:384
labels:384


In [31]:
tokenizer.decode(token_ids=collated['input_ids'], skip_special_tokens=True)

'Summarize: 10 August 2016 Last updated at 13:37 BST William Joyner was born without fingers on his left hand. The Reading FC fan s blue and white prosthetic has been made by a senior lecturer and a PHD student at the University of Bedfordshire. My friends are really excited for me, I can t wait to take it home and show everyone, he said.'

## Evaluation

Including a metric during training is often helpful for evaluating your model's performance. You can quickly load a evaluation method with the 🤗 [Evaluate](https://huggingface.co/docs/evaluate/index) library. For this task, load the [ROUGE](https://huggingface.co/spaces/evaluate-metric/rouge) metric (see the 🤗 Evaluate [quick tour](https://huggingface.co/docs/evaluate/a_quick_tour) to learn more about how to load and compute a metric):

In [32]:
rouge = evaluate.load("rouge")

Then create a function that passes your predictions and labels to [compute](https://huggingface.co/docs/evaluate/main/en/package_reference/main_classes#evaluate.EvaluationModule.compute) to calculate the ROUGE metric:

In [33]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    result = rouge.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)

    return {k: round(v, 4) for k, v in result.items()}

## Trainer API

* [TrainingArguments class](https://huggingface.co/docs/transformers/v4.27.2/en/main_classes/trainer#transformers.TrainingArguments)

The first step before we can define our Trainer is to define a TrainingArguments class that will contain all the hyperparameters the Trainer will use for training and evaluation. 

```
from transformers import TrainingArguments
training_args = TrainingArguments("bloom-trainer")
```

We can then define a Trainer by passing it all the objects constructed - the model, the training_args, the training and validation datasets, our data_collator, and our tokenizer.

* [Trainer class](https://huggingface.co/docs/transformers/main_classes/trainer)

> The ```Trainer``` class provides an API for training in **PyTorch**  for most standard use cases. It’s used in most of the [example scripts](https://github.com/huggingface/transformers/tree/main/examples). 

[TFTrainer is deprecated](https://discuss.huggingface.co/t/tensorflow-trainer/6383) for Tensorflow, and we should use Keras. See Huggingface [Tensorflow examples](https://github.com/huggingface/transformers/tree/main/examples/tensorflow) github.

> TFTrainer will be deprecated and removed in v5, we will focus on better integrating with Keras (though the means of Keras callbacks if we need to add functionality). Checkout the new [classification example](https://github.com/huggingface/transformers/blob/main/examples/tensorflow/text-classification/run_text_classification.py) for an example of where we are going.

The Trainer contains the basic training loop which supports the above features. You can subclass them and override the following methods:
```
from torch import nn
from transformers import Trainer


class CustomTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.get("labels")
        # forward pass
        outputs = model(**inputs)
        logits = outputs.get("logits")
        # compute custom loss (suppose one has 3 labels with different weights)
        loss_fct = nn.CrossEntropyLoss(weight=torch.tensor([1.0, 2.0, 3.0]))
        loss = loss_fct(logits.view(-1, self.model.config.num_labels), labels.view(-1))
        return (loss, outputs) if return_outputs else loss
```




At this point, only three steps remain:

1. Define your training hyperparameters in [Seq2SeqTrainingArguments](https://huggingface.co/docs/transformers/main/en/main_classes/trainer#transformers.Seq2SeqTrainingArguments). The only required parameter is `output_dir` which specifies where to save your model. You'll push this model to the Hub by setting `push_to_hub=True` (you need to be signed in to Hugging Face to upload your model). At the end of each epoch, the [Trainer](https://huggingface.co/docs/transformers/main/en/main_classes/trainer#transformers.Trainer) will evaluate the ROUGE metric and save the training checkpoint.
2. Pass the training arguments to [Seq2SeqTrainer](https://huggingface.co/docs/transformers/main/en/main_classes/trainer#transformers.Seq2SeqTrainer) along with the model, dataset, tokenizer, data collator, and `compute_metrics` function.
3. Call [train()](https://huggingface.co/docs/transformers/main/en/main_classes/trainer#transformers.Trainer.train) to finetune your model.

### Loss function

Use the loss function associated to the Huggingface pretrained model. No need to provide.

* [What is the loss function used in Trainer from the Transformers library of Hugging Face?](https://stackoverflow.com/a/71585375/4281353)
* [Specify Loss for Trainer / TrainingArguments](https://discuss.huggingface.co/t/specify-loss-for-trainer-trainingarguments/10481)


### max_steps for streaming dataset

* [TrainingArguments class - max_steps formula when using streaming dataset](https://discuss.huggingface.co/t/training-max-steps-formula-when-using-streaming-dataset/36531)
* [Explicitly set number of training steps using Trainer](https://discuss.huggingface.co/t/explicitly-set-number-of-training-steps-using-trainer/1127)

### num epochs for streaming dataset

* [Huge Num Epochs (9223372036854775807) when using Trainer API with streaming dataset #22757](https://github.com/huggingface/transformers/issues/22757)

### Training with streaming dataset

* [Streaming Dataset of Sequence Length 2048](https://discuss.huggingface.co/t/streaming-dataset-of-sequence-length-2048/17649)

In [34]:
# For steaming=True datasdt, *max_steps* is required to tell the total number of rows.
# https://discuss.huggingface.co/t/streaming-dataset-into-trainer-does-not-implement-len-max-steps-has-to-be-specified/32893/5
# ValueError: train_dataset does not implement __len__, max_steps has to be specified
training_args = TrainingArguments(
    output_dir="bloom_finetuned",
    max_steps=MAX_STEPS,
    num_train_epochs=-1 if DATASET_STREAMING else NUM_EPOCHS,
    per_device_train_batch_size=PER_DEVICE_BATCH_SIZE,
    per_device_eval_batch_size=PER_DEVICE_BATCH_SIZE,
    learning_rate=2e-5,
    weight_decay=0.01, 
    fp16=USE_FLOAT16,
    no_cuda=False,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=3,
    log_level="debug",
    disable_tqdm=False,
    push_to_hub=False,
)

In [35]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_validation,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

Using cuda_amp half precision backend


In [ ]:
trainer.train()
trainer.save_model("finetuned_bloom_model")

/opt/conda/lib/python3.7/site-packages/transformers/optimization.py:395: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 1,024
  Num Epochs = 3
  Instantaneous batch size per device = 1
  Total train batch size (w. parallel, distributed & accumulation) = 1
  Gradient Accumulation steps = 1
  Total optimization steps = 3,072
  Number of trainable parameters = 559,214,592


Epoch,Training Loss,Validation Loss


## Inference

In [ ]:
print(predict(prompt=prompt))

The simplest way to try out your finetuned model for inference is to use it in a [pipeline()](https://huggingface.co/docs/transformers/main/en/main_classes/pipelines#transformers.pipeline). Instantiate a `pipeline` for summarization with your model, and pass your text to it: